In [143]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import re


In [144]:
lista_carreras= ['del_Giro_de_Italia', 'del_Tour_de_Francia', 'de_la_Vuelta_a_España']


In [145]:
def escrapeo_wikipedia(lista_de_carreras):
    
    ruta_datos = '../datos/datos_wikipedia/datos_descargados'
    os.makedirs(ruta_datos, exist_ok=True)

    lista_dfs = []
    # Iteramos sobre cada una de las arreras de lista_de_carreras
    for carrera in lista_de_carreras:
        url = 'https://es.wikipedia.org/wiki/Anexo:Datos_estad%C3%ADsticos_' + carrera

        # Extraemos con Beautiful Soup la tabla de la url de arriba y la guardamos en un DataFrame
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        tabla = soup.find('table', {'class': 'wikitable'})

        # Verificamos que se ha encontrado una tabla en la página
        if tabla:
            encabezados = [encabezado.text.strip() for encabezado in tabla.find_all('th')]

            filas = []
            for fila in tabla.find_all('tr')[1:]:
                celdas = fila.find_all(['td', 'th'])
                celdas_text = [celda.text.strip() for celda in celdas]
                filas.append(celdas_text)

            df = pd.DataFrame(filas, columns=encabezados)

            # Definimos la ruta y el nombre del archivo CSV
            nombre_archivo = f'lista_dataframes[i]_{carrera}.csv'
            ruta_archivo = os.path.join(ruta_datos, nombre_archivo)

            # Guardamos el DataFrame en un archivo CSV
            df.to_csv(ruta_archivo, index=False)

            # Agregamos el DataFrame a la lista
            lista_dfs.append(df)
    
    return lista_dfs


In [146]:
[df_giro, df_tour, df_vuelta] = escrapeo_wikipedia(lista_carreras)

In [147]:
lista_dataframes=[df_giro, df_tour, df_vuelta]

In [148]:
df_giro.head()

,Edición,Año,Ganador,Tiempo del ganador,Kilómetros totales,Velocidad media (km/h),N.º de etapas[1]​,N.º de corredores que participaron,N.º de corredores que finalizaron
0,1.ª,1909,Luigi Ganna,25 pts.,2.448,"27,258",8,127,49
1,2.ª,1910,Carlo Galetti,28 pts.,"2.987,5","26,114",10,101,22
2,3.ª,1911,Carlo Galetti,50 pts.,3.452,"26,082",12,86,24
3,4.ª,1912,Atala,31 pts.,"2.439,5","27,106",8,54,26
4,5.ª,1913,Carlo Oriani,37 pts.,2.932,"26,379",9,99,35


In [149]:
df_tour.head()

,Edición,Año,Ganador,Tiempo del ganador,Kilómetros totales,Velocidad media,N.º de etapas[1]​,N.º de corredores que participaron,N.º de corredores que finalizaron
0,1,1903,Maurice Garin,94:33’14’’,2.428,"25,679 km/h",6,60,21
1,2,1904,Henri Cornet[2]​,96:05’55’’,2.428,"25,265 km/h",6,80,15
2,3,1905,Louis Trousselier,35 puntos,2.994,"25,265 km/h",11,60,24
3,4,1906,René Pottier,31 puntos,4.545,"24,463 km/h",13,82,14
4,5,1907,Lucien Petit-Breton,47 puntos,4.488,"28,740 km/h",14,93,33


In [150]:
df_vuelta.head()

,Edición,Año,Fecha(dd/mm),Ganador,Tiempo del ganador,Kilómetros totales,Velocidad media,N.º de Etapas[1]​,N.º de equipos participantes,N.º de corredores que participaron,N.º de corredores que finalizaron
0,I (1.ª),1935,29/04 - 15/05,Gustaaf Deloor,120:01’02’’,3425,"28,537 km/h.",14,-,50,29
1,II (2.ª),1936,05/05 - 31/05,Gustaaf Deloor,150:07’54’’,3407,"28,994 km/h.",21,-,50,26
2,1937-1940,Ediciones suspendidas por la Guerra civil espa...,None,None,None,None,None,None,None,None,None
3,III (3.ª),1941,12/06 - 06/07,Julián Berrendero,168:45’26’’,3406,"26,108 km/h.",22,-,32,16
4,IV (4.ª),1942,30/06 - 19/07,Julián Berrendero,134:05’09’’,3688,"27,505 km/h.",20,-,40,18


Pasamos a tipo entero la columna "Año" eliminando las filas en las que en dicha columna aparece un intervalo de años o una frase en vez de un año, pues son años en los que no se disputo la carrera, por Guerras u otros motivos.

In [151]:
for i in range(len(lista_dataframes)):
    lista_dataframes[i]['Año'] = pd.to_numeric(lista_dataframes[i]['Año'], errors='coerce')
    lista_dataframes[i] = lista_dataframes[i].dropna(subset=['Año']).copy() 
    lista_dataframes[i]['Año'] = lista_dataframes[i]['Año'].astype(int)
    lista_dataframes[i].reset_index(drop=True, inplace=True)


Hacemos que la columna "Edición" lleve una numeración sucesiva emezando en el 1, esta columna nos servirá más adelant de id para la base de datos.

In [152]:

for i in range(len(lista_dataframes)):
    lista_dataframes[i]['Edición'] = lista_dataframes[i].index + 1
    

In [153]:
lista_dataframes[2].head()

,Edición,Año,Fecha(dd/mm),Ganador,Tiempo del ganador,Kilómetros totales,Velocidad media,N.º de Etapas[1]​,N.º de equipos participantes,N.º de corredores que participaron,N.º de corredores que finalizaron
0,1,1935,29/04 - 15/05,Gustaaf Deloor,120:01’02’’,3425,"28,537 km/h.",14,-,50,29
1,2,1936,05/05 - 31/05,Gustaaf Deloor,150:07’54’’,3407,"28,994 km/h.",21,-,50,26
2,3,1941,12/06 - 06/07,Julián Berrendero,168:45’26’’,3406,"26,108 km/h.",22,-,32,16
3,4,1942,30/06 - 19/07,Julián Berrendero,134:05’09’’,3688,"27,505 km/h.",20,-,40,18
4,5,1945,10/05 - 31/05,Delio Rodríguez,135:43’55’’,3803,"28,018 km/h.",19,-,51,26


Eliminamos la columna Fecha(dd/mm) que no existe en algunos de nuestro dataFrames y queremos tenerlos todos con las mismcas columnas 

In [154]:
for i in range(len(lista_dataframes)):
    if 'Fecha(dd/mm)' in lista_dataframes[i].columns:
        lista_dataframes[i].drop(columns=['Fecha(dd/mm)'], inplace=True)

In [155]:
lista_dataframes[0].head(2)

,Edición,Año,Ganador,Tiempo del ganador,Kilómetros totales,Velocidad media (km/h),N.º de etapas[1]​,N.º de corredores que participaron,N.º de corredores que finalizaron
0,1,1909,Luigi Ganna,25 pts.,2.448,"27,258",8,127,49
1,2,1910,Carlo Galetti,28 pts.,"2.987,5","26,114",10,101,22


Creamos una nueva columna llamada 'Puntos del Ganador' donde nos llevamos los valores de la columna 'Tiempo del Ganador' que son en realidad puntos para separar aquellas ediciones que se decidieron por puntos de las que lo hicieron por tiempo. Formateamso ambas columnas

In [156]:
for i in range(len(lista_dataframes)):
    lista_dataframes[i]['Puntos del ganador'] = lista_dataframes[i]['Tiempo del ganador'].where(lista_dataframes[i]['Tiempo del ganador'].str.contains('pts|puntos', case=False, na=False))


In [157]:
for i in range(len(lista_dataframes)):
    lista_dataframes[i]['Puntos del ganador'] = lista_dataframes[i]['Puntos del ganador'].str.replace(r' pts| puntos', '', regex=True)
    lista_dataframes[i]['Puntos del ganador'] = pd.to_numeric(lista_dataframes[i]['Puntos del ganador'], errors='coerce').fillna(0).astype(int)


In [158]:
lista_dataframes[1].head()

,Edición,Año,Ganador,Tiempo del ganador,Kilómetros totales,Velocidad media,N.º de etapas[1]​,N.º de corredores que participaron,N.º de corredores que finalizaron,Puntos del ganador
0,1,1903,Maurice Garin,94:33’14’’,2.428,"25,679 km/h",6,60,21,0
1,2,1904,Henri Cornet[2]​,96:05’55’’,2.428,"25,265 km/h",6,80,15,0
2,3,1905,Louis Trousselier,35 puntos,2.994,"25,265 km/h",11,60,24,35
3,4,1906,René Pottier,31 puntos,4.545,"24,463 km/h",13,82,14,31
4,5,1907,Lucien Petit-Breton,47 puntos,4.488,"28,740 km/h",14,93,33,47


In [159]:
lista_dataframes[2].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 11 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   Edición                             79 non-null     int64 
 1   Año                                 79 non-null     int64 
 2   Ganador                             79 non-null     object
 3   Tiempo del ganador                  79 non-null     object
 4   Kilómetros totales                  79 non-null     object
 5   Velocidad media                     79 non-null     object
 6   N.º de Etapas[1]​                   79 non-null     object
 7   N.º de equipos participantes        79 non-null     object
 8   N.º de corredores que participaron  79 non-null     object
 9   N.º de corredores que finalizaron   79 non-null     object
 10  Puntos del ganador                  79 non-null     int64 
dtypes: int64(3), object(8)
memory usage: 6.9+ KB


Formateamos a float la columna "Kilómetros totales"

In [160]:
for i in range(len(lista_dataframes)):
    lista_dataframes[i]['Kilómetros totales'] = lista_dataframes[i]['Kilómetros totales'].str.replace(".", "", regex=False)
    lista_dataframes[i]['Kilómetros totales'] = lista_dataframes[i]['Kilómetros totales'].str.replace(",", ".", regex=False).astype(float)


Tratamos el caso especial de Lance Armstrong que fue declarado culpable de dopage y sus Tours fueron declarados desiertos, sin vencedor oficial

In [161]:
for i in range(len(lista_dataframes)):
    lista_dataframes[i]['Ganador'] = lista_dataframes[i]['Ganador'].replace(to_replace=r'.*Lance Armstrong.*', value='Ganador desierto', regex=True)



En el resto de casos en los que aparecen corchetes en la columna gandor que hacen referencia a notas a pie de la wikipedia eliminamos la expresión entre corchetes

In [162]:
for i in range(len(lista_dataframes)):
    lista_dataframes[i]['Ganador'] = lista_dataframes[i]['Ganador'].str.replace(r'\[.*?\]', '', regex=True).str.strip()


Borramos la columna "Numero de etapas" pues tiene caracteres espeiales y da prolbemas

In [163]:
for i in range(len(lista_dataframes)):
    lista_dataframes[i].drop(lista_dataframes[i].columns[6], axis=1, inplace=True)



In [164]:
lista_dataframes[2].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 10 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Edición                             79 non-null     int64  
 1   Año                                 79 non-null     int64  
 2   Ganador                             79 non-null     object 
 3   Tiempo del ganador                  79 non-null     object 
 4   Kilómetros totales                  79 non-null     float64
 5   Velocidad media                     79 non-null     object 
 6   N.º de equipos participantes        79 non-null     object 
 7   N.º de corredores que participaron  79 non-null     object 
 8   N.º de corredores que finalizaron   79 non-null     object 
 9   Puntos del ganador                  79 non-null     int64  
dtypes: float64(1), int64(3), object(6)
memory usage: 6.3+ KB


Formateamos la columna Tiempo del ganador

In [165]:
for i in range(len(lista_dataframes)):
    lista_dataframes[i]['Tiempo del ganador'] = lista_dataframes[i]['Tiempo del ganador'].str.replace("''", "", regex=False)
    lista_dataframes[i]['Tiempo del ganador'] = lista_dataframes[i]['Tiempo del ganador'].str.replace("'", ":", regex=False)
    lista_dataframes[i]['Tiempo del ganador'] = lista_dataframes[i]['Tiempo del ganador'].str.replace("’’", "", regex=False)
    lista_dataframes[i]['Tiempo del ganador'] = lista_dataframes[i]['Tiempo del ganador'].str.replace("’", ":", regex=False)
    lista_dataframes[i]['Tiempo del ganador'] = pd.to_timedelta(lista_dataframes[i]['Tiempo del ganador'], errors='coerce')

In [166]:
lista_dataframes[1].head()

,Edición,Año,Ganador,Tiempo del ganador,Kilómetros totales,Velocidad media,N.º de corredores que participaron,N.º de corredores que finalizaron,Puntos del ganador
0,1,1903,Maurice Garin,3 days 22:33:14,2428.0,"25,679 km/h",60,21,0
1,2,1904,Henri Cornet​,4 days 00:05:55,2428.0,"25,265 km/h",80,15,0
2,3,1905,Louis Trousselier,NaT,2994.0,"25,265 km/h",60,24,35
3,4,1906,René Pottier,NaT,4545.0,"24,463 km/h",82,14,31
4,5,1907,Lucien Petit-Breton,NaT,4488.0,"28,740 km/h",93,33,47


In [167]:

lista_dataframes[1].info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111 entries, 0 to 110
Data columns (total 9 columns):
 #   Column                              Non-Null Count  Dtype          
---  ------                              --------------  -----          
 0   Edición                             111 non-null    int64          
 1   Año                                 111 non-null    int64          
 2   Ganador                             111 non-null    object         
 3   Tiempo del ganador                  102 non-null    timedelta64[ns]
 4   Kilómetros totales                  111 non-null    float64        
 5   Velocidad media                     111 non-null    object         
 6   N.º de corredores que participaron  111 non-null    object         
 7   N.º de corredores que finalizaron   111 non-null    object         
 8   Puntos del ganador                  111 non-null    int64          
dtypes: float64(1), int64(3), object(4), timedelta64[ns](1)
memory usage: 7.9+ KB


Formateamos Velocidad media a float

In [168]:

lista_dataframes[0].info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 0 to 106
Data columns (total 9 columns):
 #   Column                              Non-Null Count  Dtype          
---  ------                              --------------  -----          
 0   Edición                             107 non-null    int64          
 1   Año                                 107 non-null    int64          
 2   Ganador                             107 non-null    object         
 3   Tiempo del ganador                  102 non-null    timedelta64[ns]
 4   Kilómetros totales                  107 non-null    float64        
 5   Velocidad media (km/h)              107 non-null    object         
 6   N.º de corredores que participaron  107 non-null    object         
 7   N.º de corredores que finalizaron   107 non-null    object         
 8   Puntos del ganador                  107 non-null    int64          
dtypes: float64(1), int64(3), object(4), timedelta64[ns](1)
memory usage: 7.7+ KB


In [169]:
# Renombrar la columna 'Velocidad media (km/h)' a 'Velocidad media'
lista_dataframes[0].rename(columns={'Velocidad media (km/h)': 'Velocidad media'}, inplace=True)



In [170]:
for i in range(len(lista_dataframes)):
    lista_dataframes[i]['Velocidad media'] = (
        lista_dataframes[i]['Velocidad media']
        .astype(str)  # Asegura que los valores son strings
        .str.replace(' km/h', '', regex=False)
        .str.replace(' km/h.', '', regex=False)
        .str.replace('km/h', '', regex=False)
    )


In [171]:
lista_dataframes[1].head()

,Edición,Año,Ganador,Tiempo del ganador,Kilómetros totales,Velocidad media,N.º de corredores que participaron,N.º de corredores que finalizaron,Puntos del ganador
0,1,1903,Maurice Garin,3 days 22:33:14,2428.0,"25,679",60,21,0
1,2,1904,Henri Cornet​,4 days 00:05:55,2428.0,"25,265",80,15,0
2,3,1905,Louis Trousselier,NaT,2994.0,"25,265",60,24,35
3,4,1906,René Pottier,NaT,4545.0,"24,463",82,14,31
4,5,1907,Lucien Petit-Breton,NaT,4488.0,"28,740",93,33,47


In [172]:
#Esta conversion ha dado muhcos problemas por el caracter especial \xa0 presente en algunas de las celdas
for i in range(len(lista_dataframes)):
    lista_dataframes[i]['Velocidad media'] = pd.to_numeric(
        lista_dataframes[i]['Velocidad media']
        .str.replace('\xa0', '')  # Elimina espacios no rompibles
        .str.replace(',', '.'),  # Reemplaza comas con puntos
        errors='coerce')

Covertimos a int las columnas que nos faltan por tratar

In [173]:
for i in range(len(lista_dataframes)):
    lista_dataframes[i][['N.º de corredores que participaron', 'N.º de corredores que finalizaron']] = lista_dataframes[i][
        ['N.º de corredores que participaron', 'N.º de corredores que finalizaron']
    ].apply(pd.to_numeric, errors='coerce').astype('Int64')



In [174]:
lista_dataframes[2].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 10 columns):
 #   Column                              Non-Null Count  Dtype          
---  ------                              --------------  -----          
 0   Edición                             79 non-null     int64          
 1   Año                                 79 non-null     int64          
 2   Ganador                             79 non-null     object         
 3   Tiempo del ganador                  79 non-null     timedelta64[ns]
 4   Kilómetros totales                  79 non-null     float64        
 5   Velocidad media                     0 non-null      float64        
 6   N.º de equipos participantes        79 non-null     object         
 7   N.º de corredores que participaron  78 non-null     Int64          
 8   N.º de corredores que finalizaron   77 non-null     Int64          
 9   Puntos del ganador                  79 non-null     int64          
dtypes: Int64(2), flo

In [175]:
lista_dataframes[2].head()

,Edición,Año,Ganador,Tiempo del ganador,Kilómetros totales,Velocidad media,N.º de equipos participantes,N.º de corredores que participaron,N.º de corredores que finalizaron,Puntos del ganador
0,1,1935,Gustaaf Deloor,5 days 00:01:02,3425.0,NaN,-,50,29,0
1,2,1936,Gustaaf Deloor,6 days 06:07:54,3407.0,NaN,-,50,26,0
2,3,1941,Julián Berrendero,7 days 00:45:26,3406.0,NaN,-,32,16,0
3,4,1942,Julián Berrendero,5 days 14:05:09,3688.0,NaN,-,40,18,0
4,5,1945,Delio Rodríguez,5 days 15:43:55,3803.0,NaN,-,51,26,0


La columna de equipos participantes solo esta en uno de los dataFrames y no aportra inforación fundamental asi que la eliminamos

In [176]:
lista_dataframes[2].drop(columns=['N.º de equipos participantes'], inplace=True)

In [177]:
lista_dataframes[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 0 to 106
Data columns (total 9 columns):
 #   Column                              Non-Null Count  Dtype          
---  ------                              --------------  -----          
 0   Edición                             107 non-null    int64          
 1   Año                                 107 non-null    int64          
 2   Ganador                             107 non-null    object         
 3   Tiempo del ganador                  102 non-null    timedelta64[ns]
 4   Kilómetros totales                  107 non-null    float64        
 5   Velocidad media                     107 non-null    float64        
 6   N.º de corredores que participaron  107 non-null    Int64          
 7   N.º de corredores que finalizaron   107 non-null    Int64          
 8   Puntos del ganador                  107 non-null    int64          
dtypes: Int64(2), float64(2), int64(3), object(1), timedelta64[ns](1)
memory usage: 7.9+ KB


In [178]:
lista_dataframes[1].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111 entries, 0 to 110
Data columns (total 9 columns):
 #   Column                              Non-Null Count  Dtype          
---  ------                              --------------  -----          
 0   Edición                             111 non-null    int64          
 1   Año                                 111 non-null    int64          
 2   Ganador                             111 non-null    object         
 3   Tiempo del ganador                  102 non-null    timedelta64[ns]
 4   Kilómetros totales                  111 non-null    float64        
 5   Velocidad media                     111 non-null    float64        
 6   N.º de corredores que participaron  107 non-null    Int64          
 7   N.º de corredores que finalizaron   103 non-null    Int64          
 8   Puntos del ganador                  111 non-null    int64          
dtypes: Int64(2), float64(2), int64(3), object(1), timedelta64[ns](1)
memory usage: 8.2+ KB


In [179]:
lista_dataframes[2].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 9 columns):
 #   Column                              Non-Null Count  Dtype          
---  ------                              --------------  -----          
 0   Edición                             79 non-null     int64          
 1   Año                                 79 non-null     int64          
 2   Ganador                             79 non-null     object         
 3   Tiempo del ganador                  79 non-null     timedelta64[ns]
 4   Kilómetros totales                  79 non-null     float64        
 5   Velocidad media                     0 non-null      float64        
 6   N.º de corredores que participaron  78 non-null     Int64          
 7   N.º de corredores que finalizaron   77 non-null     Int64          
 8   Puntos del ganador                  79 non-null     int64          
dtypes: Int64(2), float64(2), int64(3), object(1), timedelta64[ns](1)
memory usage: 5.8+ KB


In [180]:
[df_giro, df_tour, df_vuelta] = lista_dataframes

Hemos terminado con la limpieza